Code from: https://github.com/lazyprogrammer/machine_learning_examples/blob/e3b4d0bcd14c99e6b938d578802bb919018cd298/nlp_class3/cnn_toxic.py


CNNs = convolutional neural netowrk

In [14]:
# https://deeplearningcourses.com/c/deep-learning-advanced-nlp
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future

import zipfile
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from sklearn.metrics import roc_auc_score

In [5]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100 # size of each word vector
VALIDATION_SPLIT = 0.2 
BATCH_SIZE = 128
EPOCHS = 10

# Data
Kaggle Toxic Comment Classification Challenge
https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data


In [17]:
DATA_DIR = "../data"

# 1. Kaggle api installation: 
# https://github.com/Kaggle/kaggle-api

# 2. Download the data
if not os.path.exists(f"{DATA_DIR}/jigsaw-toxic-comment-classification-challenge.zip"):
    os.system(f'cd {DATA_DIR}; kaggle competitions download -c jigsaw-toxic-comment-classification-challenge')

# 3. unzip the data
path_to_zip_file = f"{DATA_DIR}/jigsaw-toxic-comment-classification-challenge.zip"
directory_to_extract_to = f'{DATA_DIR}/toxic-comment/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

# 4. list the files in the directory and unzip them
file_list = os.listdir(f"{DATA_DIR}/toxic-comment/")
for file in file_list:
    path_to_zip_file = (f"{DATA_DIR}/toxic-comment/{file}")
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

# 5. remove the zip file
os.system(f"rm {DATA_DIR}/jigsaw-toxic-comment-classification-challenge.zip")
os.system(f"rm {DATA_DIR}/toxic-comment/*.zip")


0

In [21]:
# get glove embeddings

# 1. dowload the embeddings
if not os.path.exists(f"{DATA_DIR}/glove.6B.zip"):
    os.system(f'wget -N -P {DATA_DIR} https://nlp.stanford.edu/data/glove.6B.zip')

# 2. unzip the embeddings
if not os.path.exists(f"{DATA_DIR}/glove.6B"):
    path_to_zip_file = f"{DATA_DIR}/glove.6B.zip"
    directory_to_extract_to = f"{DATA_DIR}/glove.6B/"
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

# 3. remove the zip file
os.system(f"rm {DATA_DIR}/glove.6B.zip")

0

# Load pre-trained word vectors

In [24]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open(os.path.join(f'{DATA_DIR}/glove.6B/glove.6B.{EMBEDDING_DIM}d.txt')) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 400000 word vectors.


In [23]:
# prepare text samples and their labels
print('Loading in comments...')

train = pd.read_csv("data/toxic-comment/train.csv")
sentences = train["comment_text"].fillna("DUMMY_VALUE").values
possible_labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
targets = train[possible_labels].values

Loading in comments...


FileNotFoundError: [Errno 2] File data/toxic-comment/train.csv does not exist: 'data/toxic-comment/train.csv'